In [2]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

from tqdm import tqdm
import os
import PIL.Image as Image

import torchvision
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingWarmRestarts

from model import MultiScaleCNN, AdaptiveMultiScaleCNN, Conv4Net
from evaluate import tta

device = torch.device('cuda')

from data import initialize_data, training_transforms, test_transforms # data.py in the same folder
initialize_data('images/') # extracts the zip files, makes a validation set

In [5]:
def get_data(size):
    t0,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11 = get_transforms(size)
    train_loader = torch.utils.data.DataLoader(
    torch.utils.data.ConcatDataset([
        datasets.ImageFolder('images/train_images', transform=t0),
        datasets.ImageFolder('images/train_images', transform=t1),
        datasets.ImageFolder('images/train_images', transform=t2),
        datasets.ImageFolder('images/train_images', transform=t3),
        datasets.ImageFolder('images/train_images', transform=t4),
        datasets.ImageFolder('images/train_images', transform=t5),
        datasets.ImageFolder('images/train_images', transform=t6),
        datasets.ImageFolder('images/train_images', transform=t7),
        datasets.ImageFolder('images/train_images', transform=t8),
        datasets.ImageFolder('images/train_images', transform=t9),
        datasets.ImageFolder('images/train_images', transform=t10),
    ]),
    batch_size=1024, shuffle=True, num_workers = 8)

    val_loader = torch.utils.data.DataLoader(
        datasets.ImageFolder('images/val_images', transform=t0),
        batch_size=1024, shuffle=False, num_workers = 8)
    
    return train_loader, val_loader    
    
    
# Get all the possible transforms for a given size.
def get_transforms(size):
    # Keep the same
    t0 = transforms.Compose([
        transforms.Resize((size, size)),
        transforms.ToTensor(),
        transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))
    ])

    # Scale brightness between the range (1.5,3.5)
    t1 = transforms.Compose([
        transforms.Resize((size, size)),
        transforms.ColorJitter(brightness=2.5),
        transforms.ToTensor(),
        transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))
    ])

    # Scale saturation between (1,2)
    t2 = transforms.Compose([
        transforms.Resize((size, size)),
        transforms.ColorJitter(saturation=2),
        transforms.ToTensor(),
        transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))
    ])

    # Scale contrast between (1,1.5)
    t3 = transforms.Compose([
        transforms.Resize((size, size)),
        transforms.ColorJitter(contrast=1.5),
        transforms.ToTensor(),
        transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))
    ])

    # Scale hue
    t4 = transforms.Compose([
        transforms.Resize((size, size)),
        transforms.ColorJitter(hue=0.2),
        transforms.ToTensor(),
        transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))
    ])

    # Random horizontal flips
    t5 = transforms.Compose([
        transforms.Resize((size, size)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))
    ])

    # Random shearing
    t6 = transforms.Compose([
        transforms.Resize((size, size)),
        transforms.RandomAffine(degrees=20, shear=3),
        transforms.ToTensor(),
        transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))
    ])

    # Random Translation
    t7 = transforms.Compose([
        transforms.Resize((size, size)),
        transforms.RandomAffine(degrees=10, translate=(0.2,0.2)),
        transforms.ToTensor(),
        transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))
    ])

    # Random perspective change
    t8 = transforms.Compose([
        transforms.Resize((size, size)),
        transforms.RandomPerspective(),
        transforms.ToTensor(),
        transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))
    ])

    # Random rotation
    t9 = transforms.Compose([
        transforms.Resize((size, size)),
        transforms.RandomRotation(20),
        transforms.ToTensor(),
        transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))
    ])

    # Upscale the image to size*1.5 then make a random crop of size=size
    t10 = transforms.Compose([
        transforms.Resize((int(size*1.5), int(size*1.5))),
        transforms.RandomResizedCrop(size=size),
        transforms.ToTensor(),
        transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))
    ])

    # TenCrop, only used in TTA
    t11 = transforms.Compose([
        transforms.Resize((size, size)),
        transforms.TenCrop(size),
        transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
        transforms.Lambda(lambda crops: torch.stack([transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))(crop) for crop in crops])),
    ])
    
    return t0,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11

In [6]:
def train(train_loader, epoch, model, optimizer, sched):
    model.train()
    iters = len(train_loader)
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).to(device), Variable(target).to(device)
        sched.step(epoch + batch_idx/iters)
        model.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def validation(val_loader, model):
    model.eval()
    validation_loss = 0
    correct = 0
    for data, target in val_loader:
        data, target = Variable(data).to(device), Variable(target).to(device)
        output = model(data)
        validation_loss += F.nll_loss(output, target).item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    validation_loss /= len(val_loader.dataset)
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        validation_loss, correct, len(val_loader.dataset),
        100. * correct / len(val_loader.dataset)))

In [7]:
def train_loop(model, train_dl, val_dl, optimizer, sched, epochs, exp_name):
    for epoch in range(1, epochs + 1):
        train(train_dl, epoch, model, optimizer, sched)
        validation(val_dl, model)
        if epoch%(epochs//2) == 0:
            model_file = 'Models/' + exp_name + str(epoch) + '.pth'
            torch.save(amcnn.state_dict(), model_file)
            print('\nSaved model to Models/' + model_file + '. You can run `python evaluate.py --model ' + model_file + '` to generate the Kaggle formatted csv file')

In [4]:
mcnn = MultiScaleCNN().to(device)

In [5]:
optimizer = torch.optim.Adam(mcnn.parameters())
sched = CosineAnnealingWarmRestarts(optimizer, T_0=1)
epochs = 100
exp_name = 'mcnn42_aug_500_'

In [13]:
for epoch in range(1, epochs + 1):
    train(epoch, mcnn, optimizer, sched)
    validation(mcnn)
    if epoch%50 == 0:
        model_file = exp_name + str(epoch) + '.pth'
        torch.save(mcnn.state_dict(), model_file)
        print('\nSaved model to ' + model_file + '. You can run `python evaluate.py --model ' + model_file + '` to generate the Kaggle formatted csv file')

Train Epoch: 1 [0/353390 (0%)]	Loss: 3.761759
Train Epoch: 1 [10240/353390 (3%)]	Loss: 3.409454
Train Epoch: 1 [20480/353390 (6%)]	Loss: 3.137361
Train Epoch: 1 [30720/353390 (9%)]	Loss: 2.782789
Train Epoch: 1 [40960/353390 (12%)]	Loss: 2.337990
Train Epoch: 1 [51200/353390 (14%)]	Loss: 1.992156
Train Epoch: 1 [61440/353390 (17%)]	Loss: 1.935070
Train Epoch: 1 [71680/353390 (20%)]	Loss: 1.652224
Train Epoch: 1 [81920/353390 (23%)]	Loss: 1.567037
Train Epoch: 1 [92160/353390 (26%)]	Loss: 1.433980
Train Epoch: 1 [102400/353390 (29%)]	Loss: 1.222073
Train Epoch: 1 [112640/353390 (32%)]	Loss: 1.159396
Train Epoch: 1 [122880/353390 (35%)]	Loss: 1.143262
Train Epoch: 1 [133120/353390 (38%)]	Loss: 0.989242
Train Epoch: 1 [143360/353390 (40%)]	Loss: 0.904658
Train Epoch: 1 [153600/353390 (43%)]	Loss: 0.933700
Train Epoch: 1 [163840/353390 (46%)]	Loss: 0.811260
Train Epoch: 1 [174080/353390 (49%)]	Loss: 0.772634
Train Epoch: 1 [184320/353390 (52%)]	Loss: 0.893101
Train Epoch: 1 [194560/353390 

Train Epoch: 5 [153600/353390 (43%)]	Loss: 0.279122
Train Epoch: 5 [163840/353390 (46%)]	Loss: 0.258797
Train Epoch: 5 [174080/353390 (49%)]	Loss: 0.211023
Train Epoch: 5 [184320/353390 (52%)]	Loss: 0.224969
Train Epoch: 5 [194560/353390 (55%)]	Loss: 0.261189
Train Epoch: 5 [204800/353390 (58%)]	Loss: 0.231407
Train Epoch: 5 [215040/353390 (61%)]	Loss: 0.223857
Train Epoch: 5 [225280/353390 (64%)]	Loss: 0.188491
Train Epoch: 5 [235520/353390 (66%)]	Loss: 0.242786
Train Epoch: 5 [245760/353390 (69%)]	Loss: 0.241468
Train Epoch: 5 [256000/353390 (72%)]	Loss: 0.241591
Train Epoch: 5 [266240/353390 (75%)]	Loss: 0.185447
Train Epoch: 5 [276480/353390 (78%)]	Loss: 0.204942
Train Epoch: 5 [286720/353390 (81%)]	Loss: 0.194840
Train Epoch: 5 [296960/353390 (84%)]	Loss: 0.207013
Train Epoch: 5 [307200/353390 (87%)]	Loss: 0.189093
Train Epoch: 5 [317440/353390 (90%)]	Loss: 0.184779
Train Epoch: 5 [327680/353390 (92%)]	Loss: 0.192536
Train Epoch: 5 [337920/353390 (95%)]	Loss: 0.191607
Train Epoch:

Train Epoch: 9 [296960/353390 (84%)]	Loss: 0.121347
Train Epoch: 9 [307200/353390 (87%)]	Loss: 0.154859
Train Epoch: 9 [317440/353390 (90%)]	Loss: 0.141743
Train Epoch: 9 [327680/353390 (92%)]	Loss: 0.129672
Train Epoch: 9 [337920/353390 (95%)]	Loss: 0.117214
Train Epoch: 9 [348160/353390 (98%)]	Loss: 0.166720

Validation set: Average loss: 0.0004, Accuracy: 3491/3870 (90%)

Train Epoch: 10 [0/353390 (0%)]	Loss: 0.124067
Train Epoch: 10 [10240/353390 (3%)]	Loss: 0.134843
Train Epoch: 10 [20480/353390 (6%)]	Loss: 0.172578
Train Epoch: 10 [30720/353390 (9%)]	Loss: 0.154385
Train Epoch: 10 [40960/353390 (12%)]	Loss: 0.184524
Train Epoch: 10 [51200/353390 (14%)]	Loss: 0.171599
Train Epoch: 10 [61440/353390 (17%)]	Loss: 0.136701
Train Epoch: 10 [71680/353390 (20%)]	Loss: 0.176786
Train Epoch: 10 [81920/353390 (23%)]	Loss: 0.144201
Train Epoch: 10 [92160/353390 (26%)]	Loss: 0.142849
Train Epoch: 10 [102400/353390 (29%)]	Loss: 0.197703
Train Epoch: 10 [112640/353390 (32%)]	Loss: 0.146810
Trai

Train Epoch: 14 [51200/353390 (14%)]	Loss: 0.131377
Train Epoch: 14 [61440/353390 (17%)]	Loss: 0.174380
Train Epoch: 14 [71680/353390 (20%)]	Loss: 0.170101
Train Epoch: 14 [81920/353390 (23%)]	Loss: 0.164639
Train Epoch: 14 [92160/353390 (26%)]	Loss: 0.085980
Train Epoch: 14 [102400/353390 (29%)]	Loss: 0.156272
Train Epoch: 14 [112640/353390 (32%)]	Loss: 0.142859
Train Epoch: 14 [122880/353390 (35%)]	Loss: 0.126597
Train Epoch: 14 [133120/353390 (38%)]	Loss: 0.162534
Train Epoch: 14 [143360/353390 (40%)]	Loss: 0.114805
Train Epoch: 14 [153600/353390 (43%)]	Loss: 0.151867
Train Epoch: 14 [163840/353390 (46%)]	Loss: 0.125056
Train Epoch: 14 [174080/353390 (49%)]	Loss: 0.108748
Train Epoch: 14 [184320/353390 (52%)]	Loss: 0.129540
Train Epoch: 14 [194560/353390 (55%)]	Loss: 0.156678
Train Epoch: 14 [204800/353390 (58%)]	Loss: 0.117361
Train Epoch: 14 [215040/353390 (61%)]	Loss: 0.097674
Train Epoch: 14 [225280/353390 (64%)]	Loss: 0.123985
Train Epoch: 14 [235520/353390 (66%)]	Loss: 0.10943

Train Epoch: 18 [174080/353390 (49%)]	Loss: 0.103679
Train Epoch: 18 [184320/353390 (52%)]	Loss: 0.133539
Train Epoch: 18 [194560/353390 (55%)]	Loss: 0.117440
Train Epoch: 18 [204800/353390 (58%)]	Loss: 0.101532
Train Epoch: 18 [215040/353390 (61%)]	Loss: 0.122704
Train Epoch: 18 [225280/353390 (64%)]	Loss: 0.064352
Train Epoch: 18 [235520/353390 (66%)]	Loss: 0.140830
Train Epoch: 18 [245760/353390 (69%)]	Loss: 0.115114
Train Epoch: 18 [256000/353390 (72%)]	Loss: 0.130084
Train Epoch: 18 [266240/353390 (75%)]	Loss: 0.083387
Train Epoch: 18 [276480/353390 (78%)]	Loss: 0.106254
Train Epoch: 18 [286720/353390 (81%)]	Loss: 0.102656
Train Epoch: 18 [296960/353390 (84%)]	Loss: 0.070985
Train Epoch: 18 [307200/353390 (87%)]	Loss: 0.115849
Train Epoch: 18 [317440/353390 (90%)]	Loss: 0.090796
Train Epoch: 18 [327680/353390 (92%)]	Loss: 0.098330
Train Epoch: 18 [337920/353390 (95%)]	Loss: 0.080462
Train Epoch: 18 [348160/353390 (98%)]	Loss: 0.095911

Validation set: Average loss: 0.0004, Accurac

...

Train Epoch: 91 [276480/353390 (78%)]	Loss: 0.063058
Train Epoch: 91 [286720/353390 (81%)]	Loss: 0.062263
Train Epoch: 91 [296960/353390 (84%)]	Loss: 0.069067
Train Epoch: 91 [307200/353390 (87%)]	Loss: 0.093525
Train Epoch: 91 [317440/353390 (90%)]	Loss: 0.063742
Train Epoch: 91 [327680/353390 (92%)]	Loss: 0.075293
Train Epoch: 91 [337920/353390 (95%)]	Loss: 0.043760
Train Epoch: 91 [348160/353390 (98%)]	Loss: 0.058169

Validation set: Average loss: 0.0006, Accuracy: 3597/3870 (93%)

Train Epoch: 92 [0/353390 (0%)]	Loss: 0.045997
Train Epoch: 92 [10240/353390 (3%)]	Loss: 0.082584
Train Epoch: 92 [20480/353390 (6%)]	Loss: 0.080140
Train Epoch: 92 [30720/353390 (9%)]	Loss: 0.090278
Train Epoch: 92 [40960/353390 (12%)]	Loss: 0.106918
Train Epoch: 92 [51200/353390 (14%)]	Loss: 0.069160
Train Epoch: 92 [61440/353390 (17%)]	Loss: 0.071403
Train Epoch: 92 [71680/353390 (20%)]	Loss: 0.062663
Train Epoch: 92 [81920/353390 (23%)]	Loss: 0.079787
Train Epoch: 92 [92160/353390 (26%)]	Loss: 0.07129

Train Epoch: 96 [20480/353390 (6%)]	Loss: 0.084069
Train Epoch: 96 [30720/353390 (9%)]	Loss: 0.050343
Train Epoch: 96 [40960/353390 (12%)]	Loss: 0.069965
Train Epoch: 96 [51200/353390 (14%)]	Loss: 0.109022
Train Epoch: 96 [61440/353390 (17%)]	Loss: 0.078687
Train Epoch: 96 [71680/353390 (20%)]	Loss: 0.076770
Train Epoch: 96 [81920/353390 (23%)]	Loss: 0.078574
Train Epoch: 96 [92160/353390 (26%)]	Loss: 0.071603
Train Epoch: 96 [102400/353390 (29%)]	Loss: 0.071065
Train Epoch: 96 [112640/353390 (32%)]	Loss: 0.048215
Train Epoch: 96 [122880/353390 (35%)]	Loss: 0.078153
Train Epoch: 96 [133120/353390 (38%)]	Loss: 0.071434
Train Epoch: 96 [143360/353390 (40%)]	Loss: 0.052057
Train Epoch: 96 [153600/353390 (43%)]	Loss: 0.080609
Train Epoch: 96 [163840/353390 (46%)]	Loss: 0.060719
Train Epoch: 96 [174080/353390 (49%)]	Loss: 0.060892
Train Epoch: 96 [184320/353390 (52%)]	Loss: 0.069411
Train Epoch: 96 [194560/353390 (55%)]	Loss: 0.046156
Train Epoch: 96 [204800/353390 (58%)]	Loss: 0.064954
Tra

Train Epoch: 100 [133120/353390 (38%)]	Loss: 0.082150
Train Epoch: 100 [143360/353390 (40%)]	Loss: 0.097720
Train Epoch: 100 [153600/353390 (43%)]	Loss: 0.086147
Train Epoch: 100 [163840/353390 (46%)]	Loss: 0.057840
Train Epoch: 100 [174080/353390 (49%)]	Loss: 0.054016
Train Epoch: 100 [184320/353390 (52%)]	Loss: 0.070535
Train Epoch: 100 [194560/353390 (55%)]	Loss: 0.084875
Train Epoch: 100 [204800/353390 (58%)]	Loss: 0.072168
Train Epoch: 100 [215040/353390 (61%)]	Loss: 0.098766
Train Epoch: 100 [225280/353390 (64%)]	Loss: 0.058689
Train Epoch: 100 [235520/353390 (66%)]	Loss: 0.057242
Train Epoch: 100 [245760/353390 (69%)]	Loss: 0.053656
Train Epoch: 100 [256000/353390 (72%)]	Loss: 0.100988
Train Epoch: 100 [266240/353390 (75%)]	Loss: 0.080764
Train Epoch: 100 [276480/353390 (78%)]	Loss: 0.088333
Train Epoch: 100 [286720/353390 (81%)]	Loss: 0.038742
Train Epoch: 100 [296960/353390 (84%)]	Loss: 0.060053
Train Epoch: 100 [307200/353390 (87%)]	Loss: 0.076094
Train Epoch: 100 [317440/353

Exception ignored in: <bound method Image.__del__ of <PIL.Image.Image image mode=RGB size=34x36 at 0x2B29555525C0>>
Traceback (most recent call last):
  File "/home/rm5310/pyenv/py3.6.3/lib/python3.6/site-packages/PIL/Image.py", line 634, in __del__
    self.__exit__()
  File "/home/rm5310/pyenv/py3.6.3/lib/python3.6/site-packages/PIL/Image.py", line 596, in __exit__
    if hasattr(self, "fp") and getattr(self, "_exclusive_fp", False):
KeyboardInterrupt


KeyboardInterrupt: 

## Eval

In [6]:
state_dict = torch.load('Models/mcnn42_aug_500_100.pth')
mcnn.load_state_dict(state_dict)
mcnn.eval()
exp_name = 'mcnn42_tta_aug_500_100'

In [7]:
tta(mcnn, size=32, exp_name)

100%|██████████| 12630/12630 [03:19<00:00, 63.44it/s]

Succesfully wrote mcnn42_tta_aug_500_100_out.csv, you can upload this file to the kaggle competition at https://www.kaggle.com/c/nyu-cv-fall-2018/


### Adaptive Multi-scale CNN

In [7]:
amcnn = AdaptiveMultiScaleCNN().to(device)

In [7]:
train_dl, val_dl = get_data(size=32)

In [10]:
optimizer = torch.optim.Adam(amcnn.parameters())
sched = CosineAnnealingWarmRestarts(optimizer, T_0=1)
train_loop(amcnn, train_dl, val_dl, optimizer, sched, epochs=100, exp_name='amcnn1_32_')

Train Epoch: 1 [0/388729 (0%)]	Loss: 3.760471
Train Epoch: 1 [10240/388729 (3%)]	Loss: 3.312083
Train Epoch: 1 [20480/388729 (5%)]	Loss: 2.943891
Train Epoch: 1 [30720/388729 (8%)]	Loss: 2.537194
Train Epoch: 1 [40960/388729 (11%)]	Loss: 2.042018
Train Epoch: 1 [51200/388729 (13%)]	Loss: 1.916836
Train Epoch: 1 [61440/388729 (16%)]	Loss: 1.702132
Train Epoch: 1 [71680/388729 (18%)]	Loss: 1.513314
Train Epoch: 1 [81920/388729 (21%)]	Loss: 1.301051
Train Epoch: 1 [92160/388729 (24%)]	Loss: 1.177693
Train Epoch: 1 [102400/388729 (26%)]	Loss: 1.139583
Train Epoch: 1 [112640/388729 (29%)]	Loss: 1.088615
Train Epoch: 1 [122880/388729 (32%)]	Loss: 1.005194
Train Epoch: 1 [133120/388729 (34%)]	Loss: 0.883402
Train Epoch: 1 [143360/388729 (37%)]	Loss: 0.872205
Train Epoch: 1 [153600/388729 (39%)]	Loss: 0.815859
Train Epoch: 1 [163840/388729 (42%)]	Loss: 0.850296
Train Epoch: 1 [174080/388729 (45%)]	Loss: 0.832526
Train Epoch: 1 [184320/388729 (47%)]	Loss: 0.738177
Train Epoch: 1 [194560/388729 

Train Epoch: 5 [30720/388729 (8%)]	Loss: 0.214056
Train Epoch: 5 [40960/388729 (11%)]	Loss: 0.259348
Train Epoch: 5 [51200/388729 (13%)]	Loss: 0.174840
Train Epoch: 5 [61440/388729 (16%)]	Loss: 0.224073
Train Epoch: 5 [71680/388729 (18%)]	Loss: 0.244273
Train Epoch: 5 [81920/388729 (21%)]	Loss: 0.212701
Train Epoch: 5 [92160/388729 (24%)]	Loss: 0.233353
Train Epoch: 5 [102400/388729 (26%)]	Loss: 0.244370
Train Epoch: 5 [112640/388729 (29%)]	Loss: 0.230316
Train Epoch: 5 [122880/388729 (32%)]	Loss: 0.165914
Train Epoch: 5 [133120/388729 (34%)]	Loss: 0.225929
Train Epoch: 5 [143360/388729 (37%)]	Loss: 0.172306
Train Epoch: 5 [153600/388729 (39%)]	Loss: 0.197876
Train Epoch: 5 [163840/388729 (42%)]	Loss: 0.235542
Train Epoch: 5 [174080/388729 (45%)]	Loss: 0.228447
Train Epoch: 5 [184320/388729 (47%)]	Loss: 0.225564
Train Epoch: 5 [194560/388729 (50%)]	Loss: 0.159976
Train Epoch: 5 [204800/388729 (53%)]	Loss: 0.203011
Train Epoch: 5 [215040/388729 (55%)]	Loss: 0.161763
Train Epoch: 5 [2252

Train Epoch: 9 [51200/388729 (13%)]	Loss: 0.164208
Train Epoch: 9 [61440/388729 (16%)]	Loss: 0.184634
Train Epoch: 9 [71680/388729 (18%)]	Loss: 0.172857
Train Epoch: 9 [81920/388729 (21%)]	Loss: 0.155215
Train Epoch: 9 [92160/388729 (24%)]	Loss: 0.186199
Train Epoch: 9 [102400/388729 (26%)]	Loss: 0.167272
Train Epoch: 9 [112640/388729 (29%)]	Loss: 0.161296
Train Epoch: 9 [122880/388729 (32%)]	Loss: 0.149723
Train Epoch: 9 [133120/388729 (34%)]	Loss: 0.179460
Train Epoch: 9 [143360/388729 (37%)]	Loss: 0.194668
Train Epoch: 9 [153600/388729 (39%)]	Loss: 0.132594
Train Epoch: 9 [163840/388729 (42%)]	Loss: 0.203460
Train Epoch: 9 [174080/388729 (45%)]	Loss: 0.136304
Train Epoch: 9 [184320/388729 (47%)]	Loss: 0.131623
Train Epoch: 9 [194560/388729 (50%)]	Loss: 0.120945
Train Epoch: 9 [204800/388729 (53%)]	Loss: 0.117909
Train Epoch: 9 [215040/388729 (55%)]	Loss: 0.121286
Train Epoch: 9 [225280/388729 (58%)]	Loss: 0.128196
Train Epoch: 9 [235520/388729 (61%)]	Loss: 0.114327
Train Epoch: 9 [2

Train Epoch: 13 [51200/388729 (13%)]	Loss: 0.153094
Train Epoch: 13 [61440/388729 (16%)]	Loss: 0.147665
Train Epoch: 13 [71680/388729 (18%)]	Loss: 0.113696
Train Epoch: 13 [81920/388729 (21%)]	Loss: 0.132341
Train Epoch: 13 [92160/388729 (24%)]	Loss: 0.126772
Train Epoch: 13 [102400/388729 (26%)]	Loss: 0.108625
Train Epoch: 13 [112640/388729 (29%)]	Loss: 0.151916
Train Epoch: 13 [122880/388729 (32%)]	Loss: 0.124323
Train Epoch: 13 [133120/388729 (34%)]	Loss: 0.126363
Train Epoch: 13 [143360/388729 (37%)]	Loss: 0.115259
Train Epoch: 13 [153600/388729 (39%)]	Loss: 0.145928
Train Epoch: 13 [163840/388729 (42%)]	Loss: 0.135494
Train Epoch: 13 [174080/388729 (45%)]	Loss: 0.182747
Train Epoch: 13 [184320/388729 (47%)]	Loss: 0.145543
Train Epoch: 13 [194560/388729 (50%)]	Loss: 0.094035
Train Epoch: 13 [204800/388729 (53%)]	Loss: 0.124335
Train Epoch: 13 [215040/388729 (55%)]	Loss: 0.165289
Train Epoch: 13 [225280/388729 (58%)]	Loss: 0.145782
Train Epoch: 13 [235520/388729 (61%)]	Loss: 0.12096

Train Epoch: 17 [40960/388729 (11%)]	Loss: 0.120949
Train Epoch: 17 [51200/388729 (13%)]	Loss: 0.101241
Train Epoch: 17 [61440/388729 (16%)]	Loss: 0.130994
Train Epoch: 17 [71680/388729 (18%)]	Loss: 0.153605
Train Epoch: 17 [81920/388729 (21%)]	Loss: 0.122606
Train Epoch: 17 [92160/388729 (24%)]	Loss: 0.109473
Train Epoch: 17 [102400/388729 (26%)]	Loss: 0.131444
Train Epoch: 17 [112640/388729 (29%)]	Loss: 0.156112
Train Epoch: 17 [122880/388729 (32%)]	Loss: 0.124168
Train Epoch: 17 [133120/388729 (34%)]	Loss: 0.109223
Train Epoch: 17 [143360/388729 (37%)]	Loss: 0.125324
Train Epoch: 17 [153600/388729 (39%)]	Loss: 0.102370
Train Epoch: 17 [163840/388729 (42%)]	Loss: 0.145781
Train Epoch: 17 [174080/388729 (45%)]	Loss: 0.131338
Train Epoch: 17 [184320/388729 (47%)]	Loss: 0.087870
Train Epoch: 17 [194560/388729 (50%)]	Loss: 0.104833
Train Epoch: 17 [204800/388729 (53%)]	Loss: 0.109367
Train Epoch: 17 [215040/388729 (55%)]	Loss: 0.096859
Train Epoch: 17 [225280/388729 (58%)]	Loss: 0.082107

...

Train Epoch: 84 [286720/388729 (74%)]	Loss: 0.096360
Train Epoch: 84 [296960/388729 (76%)]	Loss: 0.060092
Train Epoch: 84 [307200/388729 (79%)]	Loss: 0.043851
Train Epoch: 84 [317440/388729 (82%)]	Loss: 0.043030
Train Epoch: 84 [327680/388729 (84%)]	Loss: 0.075425
Train Epoch: 84 [337920/388729 (87%)]	Loss: 0.058338
Train Epoch: 84 [348160/388729 (89%)]	Loss: 0.051565
Train Epoch: 84 [358400/388729 (92%)]	Loss: 0.066175
Train Epoch: 84 [368640/388729 (95%)]	Loss: 0.066161
Train Epoch: 84 [378880/388729 (97%)]	Loss: 0.055007

Validation set: Average loss: 0.0006, Accuracy: 3607/3870 (93%)

Train Epoch: 85 [0/388729 (0%)]	Loss: 0.061672
Train Epoch: 85 [10240/388729 (3%)]	Loss: 0.084346
Train Epoch: 85 [20480/388729 (5%)]	Loss: 0.098541
Train Epoch: 85 [30720/388729 (8%)]	Loss: 0.127181
Train Epoch: 85 [40960/388729 (11%)]	Loss: 0.083195
Train Epoch: 85 [51200/388729 (13%)]	Loss: 0.101038
Train Epoch: 85 [61440/388729 (16%)]	Loss: 0.139286
Train Epoch: 85 [71680/388729 (18%)]	Loss: 0.049

Train Epoch: 88 [276480/388729 (71%)]	Loss: 0.060382
Train Epoch: 88 [286720/388729 (74%)]	Loss: 0.047370
Train Epoch: 88 [296960/388729 (76%)]	Loss: 0.077039
Train Epoch: 88 [307200/388729 (79%)]	Loss: 0.076388
Train Epoch: 88 [317440/388729 (82%)]	Loss: 0.042987
Train Epoch: 88 [327680/388729 (84%)]	Loss: 0.068612
Train Epoch: 88 [337920/388729 (87%)]	Loss: 0.068258
Train Epoch: 88 [348160/388729 (89%)]	Loss: 0.057946
Train Epoch: 88 [358400/388729 (92%)]	Loss: 0.079115
Train Epoch: 88 [368640/388729 (95%)]	Loss: 0.052736
Train Epoch: 88 [378880/388729 (97%)]	Loss: 0.061823

Validation set: Average loss: 0.0005, Accuracy: 3612/3870 (93%)

Train Epoch: 89 [0/388729 (0%)]	Loss: 0.049397
Train Epoch: 89 [10240/388729 (3%)]	Loss: 0.054167
Train Epoch: 89 [20480/388729 (5%)]	Loss: 0.084684
Train Epoch: 89 [30720/388729 (8%)]	Loss: 0.094529
Train Epoch: 89 [40960/388729 (11%)]	Loss: 0.104598
Train Epoch: 89 [51200/388729 (13%)]	Loss: 0.106070
Train Epoch: 89 [61440/388729 (16%)]	Loss: 0.15

Train Epoch: 92 [266240/388729 (68%)]	Loss: 0.081641
Train Epoch: 92 [276480/388729 (71%)]	Loss: 0.055454
Train Epoch: 92 [286720/388729 (74%)]	Loss: 0.100100
Train Epoch: 92 [296960/388729 (76%)]	Loss: 0.070031
Train Epoch: 92 [307200/388729 (79%)]	Loss: 0.068952
Train Epoch: 92 [317440/388729 (82%)]	Loss: 0.094572
Train Epoch: 92 [327680/388729 (84%)]	Loss: 0.079018
Train Epoch: 92 [337920/388729 (87%)]	Loss: 0.085569
Train Epoch: 92 [348160/388729 (89%)]	Loss: 0.069274
Train Epoch: 92 [358400/388729 (92%)]	Loss: 0.073925
Train Epoch: 92 [368640/388729 (95%)]	Loss: 0.048511
Train Epoch: 92 [378880/388729 (97%)]	Loss: 0.049994

Validation set: Average loss: 0.0005, Accuracy: 3607/3870 (93%)

Train Epoch: 93 [0/388729 (0%)]	Loss: 0.084458
Train Epoch: 93 [10240/388729 (3%)]	Loss: 0.042453
Train Epoch: 93 [20480/388729 (5%)]	Loss: 0.117457
Train Epoch: 93 [30720/388729 (8%)]	Loss: 0.087654
Train Epoch: 93 [40960/388729 (11%)]	Loss: 0.060907
Train Epoch: 93 [51200/388729 (13%)]	Loss: 0.0

Train Epoch: 96 [256000/388729 (66%)]	Loss: 0.052298
Train Epoch: 96 [266240/388729 (68%)]	Loss: 0.036345
Train Epoch: 96 [276480/388729 (71%)]	Loss: 0.112967
Train Epoch: 96 [286720/388729 (74%)]	Loss: 0.072102
Train Epoch: 96 [296960/388729 (76%)]	Loss: 0.080528
Train Epoch: 96 [307200/388729 (79%)]	Loss: 0.076018
Train Epoch: 96 [317440/388729 (82%)]	Loss: 0.068838
Train Epoch: 96 [327680/388729 (84%)]	Loss: 0.035230
Train Epoch: 96 [337920/388729 (87%)]	Loss: 0.052491
Train Epoch: 96 [348160/388729 (89%)]	Loss: 0.080030
Train Epoch: 96 [358400/388729 (92%)]	Loss: 0.060772
Train Epoch: 96 [368640/388729 (95%)]	Loss: 0.074080
Train Epoch: 96 [378880/388729 (97%)]	Loss: 0.049018

Validation set: Average loss: 0.0006, Accuracy: 3592/3870 (93%)

Train Epoch: 97 [0/388729 (0%)]	Loss: 0.059631
Train Epoch: 97 [10240/388729 (3%)]	Loss: 0.090693
Train Epoch: 97 [20480/388729 (5%)]	Loss: 0.066129
Train Epoch: 97 [30720/388729 (8%)]	Loss: 0.045389
Train Epoch: 97 [40960/388729 (11%)]	Loss: 0.

Train Epoch: 100 [245760/388729 (63%)]	Loss: 0.072453
Train Epoch: 100 [256000/388729 (66%)]	Loss: 0.083671
Train Epoch: 100 [266240/388729 (68%)]	Loss: 0.074733
Train Epoch: 100 [276480/388729 (71%)]	Loss: 0.071155
Train Epoch: 100 [286720/388729 (74%)]	Loss: 0.071529
Train Epoch: 100 [296960/388729 (76%)]	Loss: 0.047407
Train Epoch: 100 [307200/388729 (79%)]	Loss: 0.042734
Train Epoch: 100 [317440/388729 (82%)]	Loss: 0.057377
Train Epoch: 100 [327680/388729 (84%)]	Loss: 0.066010
Train Epoch: 100 [337920/388729 (87%)]	Loss: 0.060208
Train Epoch: 100 [348160/388729 (89%)]	Loss: 0.078879
Train Epoch: 100 [358400/388729 (92%)]	Loss: 0.044983
Train Epoch: 100 [368640/388729 (95%)]	Loss: 0.058954
Train Epoch: 100 [378880/388729 (97%)]	Loss: 0.074505

Validation set: Average loss: 0.0005, Accuracy: 3637/3870 (94%)


Saved model to Models/Models/amcnn1_32_100.pth. You can run `python evaluate.py --model Models/amcnn1_32_100.pth` to generate the Kaggle formatted csv file


In [11]:
state_dict = torch.load('Models/amcnn1_32_100.pth')
amcnn.load_state_dict(state_dict)

<All keys matched successfully>

In [12]:
train_dl, val_dl = get_data(size=50)

In [13]:
train_loop(amcnn, train_dl, val_dl, optimizer, sched, epochs=50, exp_name='amcnn1_50_')

Train Epoch: 1 [0/388729 (0%)]	Loss: 0.090048
Train Epoch: 1 [10240/388729 (3%)]	Loss: 0.088581
Train Epoch: 1 [20480/388729 (5%)]	Loss: 0.128636
Train Epoch: 1 [30720/388729 (8%)]	Loss: 0.094943
Train Epoch: 1 [40960/388729 (11%)]	Loss: 0.074820
Train Epoch: 1 [51200/388729 (13%)]	Loss: 0.099027
Train Epoch: 1 [61440/388729 (16%)]	Loss: 0.069364
Train Epoch: 1 [71680/388729 (18%)]	Loss: 0.074618
Train Epoch: 1 [81920/388729 (21%)]	Loss: 0.074193
Train Epoch: 1 [92160/388729 (24%)]	Loss: 0.067914
Train Epoch: 1 [102400/388729 (26%)]	Loss: 0.123903
Train Epoch: 1 [112640/388729 (29%)]	Loss: 0.118425
Train Epoch: 1 [122880/388729 (32%)]	Loss: 0.100210
Train Epoch: 1 [133120/388729 (34%)]	Loss: 0.084047
Train Epoch: 1 [143360/388729 (37%)]	Loss: 0.060777
Train Epoch: 1 [153600/388729 (39%)]	Loss: 0.073014
Train Epoch: 1 [163840/388729 (42%)]	Loss: 0.074101
Train Epoch: 1 [174080/388729 (45%)]	Loss: 0.102725
Train Epoch: 1 [184320/388729 (47%)]	Loss: 0.092703
Train Epoch: 1 [194560/388729 

Train Epoch: 5 [30720/388729 (8%)]	Loss: 0.093044
Train Epoch: 5 [40960/388729 (11%)]	Loss: 0.037824
Train Epoch: 5 [51200/388729 (13%)]	Loss: 0.077815
Train Epoch: 5 [61440/388729 (16%)]	Loss: 0.078078
Train Epoch: 5 [71680/388729 (18%)]	Loss: 0.069716
Train Epoch: 5 [81920/388729 (21%)]	Loss: 0.085698
Train Epoch: 5 [92160/388729 (24%)]	Loss: 0.083646
Train Epoch: 5 [102400/388729 (26%)]	Loss: 0.091593
Train Epoch: 5 [112640/388729 (29%)]	Loss: 0.042254
Train Epoch: 5 [122880/388729 (32%)]	Loss: 0.074733
Train Epoch: 5 [133120/388729 (34%)]	Loss: 0.062654
Train Epoch: 5 [143360/388729 (37%)]	Loss: 0.059819
Train Epoch: 5 [153600/388729 (39%)]	Loss: 0.054989
Train Epoch: 5 [163840/388729 (42%)]	Loss: 0.063916
Train Epoch: 5 [174080/388729 (45%)]	Loss: 0.097852
Train Epoch: 5 [184320/388729 (47%)]	Loss: 0.080353
Train Epoch: 5 [194560/388729 (50%)]	Loss: 0.065202
Train Epoch: 5 [204800/388729 (53%)]	Loss: 0.036959
Train Epoch: 5 [215040/388729 (55%)]	Loss: 0.063052
Train Epoch: 5 [2252

Train Epoch: 9 [51200/388729 (13%)]	Loss: 0.022023
Train Epoch: 9 [61440/388729 (16%)]	Loss: 0.104855
Train Epoch: 9 [71680/388729 (18%)]	Loss: 0.066316
Train Epoch: 9 [81920/388729 (21%)]	Loss: 0.070834
Train Epoch: 9 [92160/388729 (24%)]	Loss: 0.097915
Train Epoch: 9 [102400/388729 (26%)]	Loss: 0.053857
Train Epoch: 9 [112640/388729 (29%)]	Loss: 0.043290
Train Epoch: 9 [122880/388729 (32%)]	Loss: 0.111109
Train Epoch: 9 [133120/388729 (34%)]	Loss: 0.050515
Train Epoch: 9 [143360/388729 (37%)]	Loss: 0.053659
Train Epoch: 9 [153600/388729 (39%)]	Loss: 0.048848
Train Epoch: 9 [163840/388729 (42%)]	Loss: 0.040577
Train Epoch: 9 [174080/388729 (45%)]	Loss: 0.055644
Train Epoch: 9 [184320/388729 (47%)]	Loss: 0.095097
Train Epoch: 9 [194560/388729 (50%)]	Loss: 0.050169
Train Epoch: 9 [204800/388729 (53%)]	Loss: 0.065311
Train Epoch: 9 [215040/388729 (55%)]	Loss: 0.070302
Train Epoch: 9 [225280/388729 (58%)]	Loss: 0.062300
Train Epoch: 9 [235520/388729 (61%)]	Loss: 0.032549
Train Epoch: 9 [2

Train Epoch: 13 [51200/388729 (13%)]	Loss: 0.069703
Train Epoch: 13 [61440/388729 (16%)]	Loss: 0.061510
Train Epoch: 13 [71680/388729 (18%)]	Loss: 0.085314
Train Epoch: 13 [81920/388729 (21%)]	Loss: 0.091978
Train Epoch: 13 [92160/388729 (24%)]	Loss: 0.086850
Train Epoch: 13 [102400/388729 (26%)]	Loss: 0.072715
Train Epoch: 13 [112640/388729 (29%)]	Loss: 0.062684
Train Epoch: 13 [122880/388729 (32%)]	Loss: 0.082733
Train Epoch: 13 [133120/388729 (34%)]	Loss: 0.063801
Train Epoch: 13 [143360/388729 (37%)]	Loss: 0.041483
Train Epoch: 13 [153600/388729 (39%)]	Loss: 0.050561
Train Epoch: 13 [163840/388729 (42%)]	Loss: 0.059701
Train Epoch: 13 [174080/388729 (45%)]	Loss: 0.062869
Train Epoch: 13 [184320/388729 (47%)]	Loss: 0.065345
Train Epoch: 13 [194560/388729 (50%)]	Loss: 0.071895
Train Epoch: 13 [204800/388729 (53%)]	Loss: 0.066864
Train Epoch: 13 [215040/388729 (55%)]	Loss: 0.054151
Train Epoch: 13 [225280/388729 (58%)]	Loss: 0.070022
Train Epoch: 13 [235520/388729 (61%)]	Loss: 0.07926

Train Epoch: 17 [40960/388729 (11%)]	Loss: 0.083454
Train Epoch: 17 [51200/388729 (13%)]	Loss: 0.099508
Train Epoch: 17 [61440/388729 (16%)]	Loss: 0.060154
Train Epoch: 17 [71680/388729 (18%)]	Loss: 0.055037
Train Epoch: 17 [81920/388729 (21%)]	Loss: 0.076965
Train Epoch: 17 [92160/388729 (24%)]	Loss: 0.082613
Train Epoch: 17 [102400/388729 (26%)]	Loss: 0.084666
Train Epoch: 17 [112640/388729 (29%)]	Loss: 0.056566
Train Epoch: 17 [122880/388729 (32%)]	Loss: 0.084370
Train Epoch: 17 [133120/388729 (34%)]	Loss: 0.087070
Train Epoch: 17 [143360/388729 (37%)]	Loss: 0.081538
Train Epoch: 17 [153600/388729 (39%)]	Loss: 0.058346
Train Epoch: 17 [163840/388729 (42%)]	Loss: 0.054813
Train Epoch: 17 [174080/388729 (45%)]	Loss: 0.036335
Train Epoch: 17 [184320/388729 (47%)]	Loss: 0.099920
Train Epoch: 17 [194560/388729 (50%)]	Loss: 0.062224
Train Epoch: 17 [204800/388729 (53%)]	Loss: 0.079852
Train Epoch: 17 [215040/388729 (55%)]	Loss: 0.063922
Train Epoch: 17 [225280/388729 (58%)]	Loss: 0.047364

...

Train Epoch: 32 [378880/388729 (97%)]	Loss: 0.039950

Validation set: Average loss: 0.0006, Accuracy: 3654/3870 (94%)

Train Epoch: 33 [0/388729 (0%)]	Loss: 0.042433
Train Epoch: 33 [10240/388729 (3%)]	Loss: 0.035234
Train Epoch: 33 [20480/388729 (5%)]	Loss: 0.041891
Train Epoch: 33 [30720/388729 (8%)]	Loss: 0.022026
Train Epoch: 33 [40960/388729 (11%)]	Loss: 0.054428
Train Epoch: 33 [51200/388729 (13%)]	Loss: 0.062978
Train Epoch: 33 [61440/388729 (16%)]	Loss: 0.058892
Train Epoch: 33 [71680/388729 (18%)]	Loss: 0.065980
Train Epoch: 33 [81920/388729 (21%)]	Loss: 0.088358
Train Epoch: 33 [92160/388729 (24%)]	Loss: 0.067322
Train Epoch: 33 [102400/388729 (26%)]	Loss: 0.051133
Train Epoch: 33 [112640/388729 (29%)]	Loss: 0.107714
Train Epoch: 33 [122880/388729 (32%)]	Loss: 0.063980
Train Epoch: 33 [133120/388729 (34%)]	Loss: 0.061295
Train Epoch: 33 [143360/388729 (37%)]	Loss: 0.084044
Train Epoch: 33 [153600/388729 (39%)]	Loss: 0.030855
Train Epoch: 33 [163840/388729 (42%)]	Loss: 0.05244

Train Epoch: 36 [368640/388729 (95%)]	Loss: 0.055792
Train Epoch: 36 [378880/388729 (97%)]	Loss: 0.058830

Validation set: Average loss: 0.0007, Accuracy: 3629/3870 (94%)

Train Epoch: 37 [0/388729 (0%)]	Loss: 0.056218
Train Epoch: 37 [10240/388729 (3%)]	Loss: 0.076590
Train Epoch: 37 [20480/388729 (5%)]	Loss: 0.081427
Train Epoch: 37 [30720/388729 (8%)]	Loss: 0.046339
Train Epoch: 37 [40960/388729 (11%)]	Loss: 0.071527
Train Epoch: 37 [51200/388729 (13%)]	Loss: 0.047744
Train Epoch: 37 [61440/388729 (16%)]	Loss: 0.042493
Train Epoch: 37 [71680/388729 (18%)]	Loss: 0.109096
Train Epoch: 37 [81920/388729 (21%)]	Loss: 0.069071
Train Epoch: 37 [92160/388729 (24%)]	Loss: 0.121221
Train Epoch: 37 [102400/388729 (26%)]	Loss: 0.069059
Train Epoch: 37 [112640/388729 (29%)]	Loss: 0.131416
Train Epoch: 37 [122880/388729 (32%)]	Loss: 0.097932
Train Epoch: 37 [133120/388729 (34%)]	Loss: 0.087803
Train Epoch: 37 [143360/388729 (37%)]	Loss: 0.074366
Train Epoch: 37 [153600/388729 (39%)]	Loss: 0.05478

Train Epoch: 40 [358400/388729 (92%)]	Loss: 0.055450
Train Epoch: 40 [368640/388729 (95%)]	Loss: 0.035608
Train Epoch: 40 [378880/388729 (97%)]	Loss: 0.038257

Validation set: Average loss: 0.0006, Accuracy: 3610/3870 (93%)

Train Epoch: 41 [0/388729 (0%)]	Loss: 0.072118
Train Epoch: 41 [10240/388729 (3%)]	Loss: 0.023993
Train Epoch: 41 [20480/388729 (5%)]	Loss: 0.097697
Train Epoch: 41 [30720/388729 (8%)]	Loss: 0.070068
Train Epoch: 41 [40960/388729 (11%)]	Loss: 0.065563
Train Epoch: 41 [51200/388729 (13%)]	Loss: 0.056889
Train Epoch: 41 [61440/388729 (16%)]	Loss: 0.082913
Train Epoch: 41 [71680/388729 (18%)]	Loss: 0.058185
Train Epoch: 41 [81920/388729 (21%)]	Loss: 0.055719
Train Epoch: 41 [92160/388729 (24%)]	Loss: 0.081384
Train Epoch: 41 [102400/388729 (26%)]	Loss: 0.054557
Train Epoch: 41 [112640/388729 (29%)]	Loss: 0.047047
Train Epoch: 41 [122880/388729 (32%)]	Loss: 0.052980
Train Epoch: 41 [133120/388729 (34%)]	Loss: 0.068297
Train Epoch: 41 [143360/388729 (37%)]	Loss: 0.05520

Train Epoch: 44 [348160/388729 (89%)]	Loss: 0.047547
Train Epoch: 44 [358400/388729 (92%)]	Loss: 0.047369
Train Epoch: 44 [368640/388729 (95%)]	Loss: 0.036079
Train Epoch: 44 [378880/388729 (97%)]	Loss: 0.091178

Validation set: Average loss: 0.0006, Accuracy: 3613/3870 (93%)

Train Epoch: 45 [0/388729 (0%)]	Loss: 0.055237
Train Epoch: 45 [10240/388729 (3%)]	Loss: 0.192330
Train Epoch: 45 [20480/388729 (5%)]	Loss: 0.070595
Train Epoch: 45 [30720/388729 (8%)]	Loss: 0.061014
Train Epoch: 45 [40960/388729 (11%)]	Loss: 0.061600
Train Epoch: 45 [51200/388729 (13%)]	Loss: 0.052179
Train Epoch: 45 [61440/388729 (16%)]	Loss: 0.079256
Train Epoch: 45 [71680/388729 (18%)]	Loss: 0.068207
Train Epoch: 45 [81920/388729 (21%)]	Loss: 0.062357
Train Epoch: 45 [92160/388729 (24%)]	Loss: 0.050930
Train Epoch: 45 [102400/388729 (26%)]	Loss: 0.066338
Train Epoch: 45 [112640/388729 (29%)]	Loss: 0.063066
Train Epoch: 45 [122880/388729 (32%)]	Loss: 0.061939
Train Epoch: 45 [133120/388729 (34%)]	Loss: 0.05970

Train Epoch: 48 [337920/388729 (87%)]	Loss: 0.038870
Train Epoch: 48 [348160/388729 (89%)]	Loss: 0.043915
Train Epoch: 48 [358400/388729 (92%)]	Loss: 0.056181
Train Epoch: 48 [368640/388729 (95%)]	Loss: 0.009135
Train Epoch: 48 [378880/388729 (97%)]	Loss: 0.036283

Validation set: Average loss: 0.0007, Accuracy: 3619/3870 (94%)

Train Epoch: 49 [0/388729 (0%)]	Loss: 0.027288
Train Epoch: 49 [10240/388729 (3%)]	Loss: 0.057326
Train Epoch: 49 [20480/388729 (5%)]	Loss: 0.088928
Train Epoch: 49 [30720/388729 (8%)]	Loss: 0.068172
Train Epoch: 49 [40960/388729 (11%)]	Loss: 0.062595
Train Epoch: 49 [51200/388729 (13%)]	Loss: 0.070137
Train Epoch: 49 [61440/388729 (16%)]	Loss: 0.065463
Train Epoch: 49 [71680/388729 (18%)]	Loss: 0.062757
Train Epoch: 49 [81920/388729 (21%)]	Loss: 0.052612
Train Epoch: 49 [92160/388729 (24%)]	Loss: 0.065398
Train Epoch: 49 [102400/388729 (26%)]	Loss: 0.050531
Train Epoch: 49 [112640/388729 (29%)]	Loss: 0.080320
Train Epoch: 49 [122880/388729 (32%)]	Loss: 0.05875

In [14]:
state_dict = torch.load('Models/amcnn1_50_50.pth')
amcnn.load_state_dict(state_dict)

<All keys matched successfully>

In [15]:
train_dl, val_dl = get_data(size=70)

In [16]:
train_loop(amcnn, train_dl, val_dl, optimizer, sched, epochs=50, exp_name='amcnn1_70_')

Train Epoch: 1 [0/388729 (0%)]	Loss: 0.052230
Train Epoch: 1 [10240/388729 (3%)]	Loss: 0.057504
Train Epoch: 1 [20480/388729 (5%)]	Loss: 0.055668
Train Epoch: 1 [30720/388729 (8%)]	Loss: 0.064176
Train Epoch: 1 [40960/388729 (11%)]	Loss: 0.068172
Train Epoch: 1 [51200/388729 (13%)]	Loss: 0.091435
Train Epoch: 1 [61440/388729 (16%)]	Loss: 0.082223
Train Epoch: 1 [71680/388729 (18%)]	Loss: 0.060050
Train Epoch: 1 [81920/388729 (21%)]	Loss: 0.070539
Train Epoch: 1 [92160/388729 (24%)]	Loss: 0.067857
Train Epoch: 1 [102400/388729 (26%)]	Loss: 0.082673
Train Epoch: 1 [112640/388729 (29%)]	Loss: 0.053731
Train Epoch: 1 [122880/388729 (32%)]	Loss: 0.066060
Train Epoch: 1 [133120/388729 (34%)]	Loss: 0.072274
Train Epoch: 1 [143360/388729 (37%)]	Loss: 0.058351
Train Epoch: 1 [153600/388729 (39%)]	Loss: 0.075673
Train Epoch: 1 [163840/388729 (42%)]	Loss: 0.047871
Train Epoch: 1 [174080/388729 (45%)]	Loss: 0.045336
Train Epoch: 1 [184320/388729 (47%)]	Loss: 0.053851
Train Epoch: 1 [194560/388729 

Train Epoch: 5 [30720/388729 (8%)]	Loss: 0.073904
Train Epoch: 5 [40960/388729 (11%)]	Loss: 0.050418
Train Epoch: 5 [51200/388729 (13%)]	Loss: 0.061128
Train Epoch: 5 [61440/388729 (16%)]	Loss: 0.066477
Train Epoch: 5 [71680/388729 (18%)]	Loss: 0.043233
Train Epoch: 5 [81920/388729 (21%)]	Loss: 0.039805
Train Epoch: 5 [92160/388729 (24%)]	Loss: 0.095895
Train Epoch: 5 [102400/388729 (26%)]	Loss: 0.119685
Train Epoch: 5 [112640/388729 (29%)]	Loss: 0.051156
Train Epoch: 5 [122880/388729 (32%)]	Loss: 0.078926
Train Epoch: 5 [133120/388729 (34%)]	Loss: 0.079197
Train Epoch: 5 [143360/388729 (37%)]	Loss: 0.066118
Train Epoch: 5 [153600/388729 (39%)]	Loss: 0.050466
Train Epoch: 5 [163840/388729 (42%)]	Loss: 0.074733
Train Epoch: 5 [174080/388729 (45%)]	Loss: 0.071100
Train Epoch: 5 [184320/388729 (47%)]	Loss: 0.054574
Train Epoch: 5 [194560/388729 (50%)]	Loss: 0.105084
Train Epoch: 5 [204800/388729 (53%)]	Loss: 0.063973
Train Epoch: 5 [215040/388729 (55%)]	Loss: 0.050183
Train Epoch: 5 [2252

Train Epoch: 9 [51200/388729 (13%)]	Loss: 0.085022
Train Epoch: 9 [61440/388729 (16%)]	Loss: 0.083054
Train Epoch: 9 [71680/388729 (18%)]	Loss: 0.147304
Train Epoch: 9 [81920/388729 (21%)]	Loss: 0.144802
Train Epoch: 9 [92160/388729 (24%)]	Loss: 0.102307
Train Epoch: 9 [102400/388729 (26%)]	Loss: 0.106943
Train Epoch: 9 [112640/388729 (29%)]	Loss: 0.147563
Train Epoch: 9 [122880/388729 (32%)]	Loss: 0.096581
Train Epoch: 9 [133120/388729 (34%)]	Loss: 0.222590
Train Epoch: 9 [143360/388729 (37%)]	Loss: 0.098620
Train Epoch: 9 [153600/388729 (39%)]	Loss: 0.073200
Train Epoch: 9 [163840/388729 (42%)]	Loss: 0.055141
Train Epoch: 9 [174080/388729 (45%)]	Loss: 0.085301
Train Epoch: 9 [184320/388729 (47%)]	Loss: 0.044476
Train Epoch: 9 [194560/388729 (50%)]	Loss: 0.085225
Train Epoch: 9 [204800/388729 (53%)]	Loss: 0.092209
Train Epoch: 9 [215040/388729 (55%)]	Loss: 0.110561
Train Epoch: 9 [225280/388729 (58%)]	Loss: 0.097154
Train Epoch: 9 [235520/388729 (61%)]	Loss: 0.065066
Train Epoch: 9 [2

Train Epoch: 13 [51200/388729 (13%)]	Loss: 0.068513
Train Epoch: 13 [61440/388729 (16%)]	Loss: 0.052581
Train Epoch: 13 [71680/388729 (18%)]	Loss: 0.069907
Train Epoch: 13 [81920/388729 (21%)]	Loss: 0.075790
Train Epoch: 13 [92160/388729 (24%)]	Loss: 0.065410
Train Epoch: 13 [102400/388729 (26%)]	Loss: 0.077958
Train Epoch: 13 [112640/388729 (29%)]	Loss: 0.062310
Train Epoch: 13 [122880/388729 (32%)]	Loss: 0.068794
Train Epoch: 13 [133120/388729 (34%)]	Loss: 0.075335
Train Epoch: 13 [143360/388729 (37%)]	Loss: 0.047701
Train Epoch: 13 [153600/388729 (39%)]	Loss: 0.061112
Train Epoch: 13 [163840/388729 (42%)]	Loss: 0.052994
Train Epoch: 13 [174080/388729 (45%)]	Loss: 0.056710
Train Epoch: 13 [184320/388729 (47%)]	Loss: 0.027234
Train Epoch: 13 [194560/388729 (50%)]	Loss: 0.058883
Train Epoch: 13 [204800/388729 (53%)]	Loss: 0.064885
Train Epoch: 13 [215040/388729 (55%)]	Loss: 0.054585
Train Epoch: 13 [225280/388729 (58%)]	Loss: 0.060017
Train Epoch: 13 [235520/388729 (61%)]	Loss: 0.05708

Train Epoch: 17 [40960/388729 (11%)]	Loss: 0.098210
Train Epoch: 17 [51200/388729 (13%)]	Loss: 0.053820
Train Epoch: 17 [61440/388729 (16%)]	Loss: 0.083390
Train Epoch: 17 [71680/388729 (18%)]	Loss: 0.065745
Train Epoch: 17 [81920/388729 (21%)]	Loss: 0.051559
Train Epoch: 17 [92160/388729 (24%)]	Loss: 0.034436
Train Epoch: 17 [102400/388729 (26%)]	Loss: 0.036941
Train Epoch: 17 [112640/388729 (29%)]	Loss: 0.053481
Train Epoch: 17 [122880/388729 (32%)]	Loss: 0.048304
Train Epoch: 17 [133120/388729 (34%)]	Loss: 0.062294
Train Epoch: 17 [143360/388729 (37%)]	Loss: 0.067457
Train Epoch: 17 [153600/388729 (39%)]	Loss: 0.039951
Train Epoch: 17 [163840/388729 (42%)]	Loss: 0.079978
Train Epoch: 17 [174080/388729 (45%)]	Loss: 0.049658
Train Epoch: 17 [184320/388729 (47%)]	Loss: 0.048634
Train Epoch: 17 [194560/388729 (50%)]	Loss: 0.056541
Train Epoch: 17 [204800/388729 (53%)]	Loss: 0.037536
Train Epoch: 17 [215040/388729 (55%)]	Loss: 0.053236
Train Epoch: 17 [225280/388729 (58%)]	Loss: 0.046828

...

Train Epoch: 32 [378880/388729 (97%)]	Loss: 0.026883

Validation set: Average loss: 0.0007, Accuracy: 3612/3870 (93%)

Train Epoch: 33 [0/388729 (0%)]	Loss: 0.044545
Train Epoch: 33 [10240/388729 (3%)]	Loss: 0.041685
Train Epoch: 33 [20480/388729 (5%)]	Loss: 0.050817
Train Epoch: 33 [30720/388729 (8%)]	Loss: 0.081934
Train Epoch: 33 [40960/388729 (11%)]	Loss: 0.064168
Train Epoch: 33 [51200/388729 (13%)]	Loss: 0.107162
Train Epoch: 33 [61440/388729 (16%)]	Loss: 0.089266
Train Epoch: 33 [71680/388729 (18%)]	Loss: 0.058519
Train Epoch: 33 [81920/388729 (21%)]	Loss: 0.113423
Train Epoch: 33 [92160/388729 (24%)]	Loss: 0.070922
Train Epoch: 33 [102400/388729 (26%)]	Loss: 0.046902
Train Epoch: 33 [112640/388729 (29%)]	Loss: 0.062624
Train Epoch: 33 [122880/388729 (32%)]	Loss: 0.051543
Train Epoch: 33 [133120/388729 (34%)]	Loss: 0.076492
Train Epoch: 33 [143360/388729 (37%)]	Loss: 0.070312
Train Epoch: 33 [153600/388729 (39%)]	Loss: 0.067824
Train Epoch: 33 [163840/388729 (42%)]	Loss: 0.05756

Train Epoch: 36 [368640/388729 (95%)]	Loss: 0.030064
Train Epoch: 36 [378880/388729 (97%)]	Loss: 0.049120

Validation set: Average loss: 0.0008, Accuracy: 3601/3870 (93%)

Train Epoch: 37 [0/388729 (0%)]	Loss: 0.034500
Train Epoch: 37 [10240/388729 (3%)]	Loss: 0.055477
Train Epoch: 37 [20480/388729 (5%)]	Loss: 0.076605
Train Epoch: 37 [30720/388729 (8%)]	Loss: 0.054539
Train Epoch: 37 [40960/388729 (11%)]	Loss: 0.052034
Train Epoch: 37 [51200/388729 (13%)]	Loss: 0.093758
Train Epoch: 37 [61440/388729 (16%)]	Loss: 0.054355
Train Epoch: 37 [71680/388729 (18%)]	Loss: 0.060512
Train Epoch: 37 [81920/388729 (21%)]	Loss: 0.056571
Train Epoch: 37 [92160/388729 (24%)]	Loss: 0.065628
Train Epoch: 37 [102400/388729 (26%)]	Loss: 0.071738
Train Epoch: 37 [112640/388729 (29%)]	Loss: 0.040973
Train Epoch: 37 [122880/388729 (32%)]	Loss: 0.122253
Train Epoch: 37 [133120/388729 (34%)]	Loss: 0.063329
Train Epoch: 37 [143360/388729 (37%)]	Loss: 0.083364
Train Epoch: 37 [153600/388729 (39%)]	Loss: 0.08425

Train Epoch: 40 [358400/388729 (92%)]	Loss: 0.048573
Train Epoch: 40 [368640/388729 (95%)]	Loss: 0.050735
Train Epoch: 40 [378880/388729 (97%)]	Loss: 0.078551

Validation set: Average loss: 0.0007, Accuracy: 3613/3870 (93%)

Train Epoch: 41 [0/388729 (0%)]	Loss: 0.035080
Train Epoch: 41 [10240/388729 (3%)]	Loss: 0.048684
Train Epoch: 41 [20480/388729 (5%)]	Loss: 0.065196
Train Epoch: 41 [30720/388729 (8%)]	Loss: 0.082848
Train Epoch: 41 [40960/388729 (11%)]	Loss: 0.071864
Train Epoch: 41 [51200/388729 (13%)]	Loss: 0.086855
Train Epoch: 41 [61440/388729 (16%)]	Loss: 0.036866
Train Epoch: 41 [71680/388729 (18%)]	Loss: 0.066480
Train Epoch: 41 [81920/388729 (21%)]	Loss: 0.057312
Train Epoch: 41 [92160/388729 (24%)]	Loss: 0.073566
Train Epoch: 41 [102400/388729 (26%)]	Loss: 0.098467
Train Epoch: 41 [112640/388729 (29%)]	Loss: 0.077698
Train Epoch: 41 [122880/388729 (32%)]	Loss: 0.046723
Train Epoch: 41 [133120/388729 (34%)]	Loss: 0.092380
Train Epoch: 41 [143360/388729 (37%)]	Loss: 0.05017

Train Epoch: 44 [348160/388729 (89%)]	Loss: 0.038729
Train Epoch: 44 [358400/388729 (92%)]	Loss: 0.079383
Train Epoch: 44 [368640/388729 (95%)]	Loss: 0.029318
Train Epoch: 44 [378880/388729 (97%)]	Loss: 0.049042

Validation set: Average loss: 0.0007, Accuracy: 3639/3870 (94%)

Train Epoch: 45 [0/388729 (0%)]	Loss: 0.035417
Train Epoch: 45 [10240/388729 (3%)]	Loss: 0.050564
Train Epoch: 45 [20480/388729 (5%)]	Loss: 0.026874
Train Epoch: 45 [30720/388729 (8%)]	Loss: 0.090262
Train Epoch: 45 [40960/388729 (11%)]	Loss: 0.057648
Train Epoch: 45 [51200/388729 (13%)]	Loss: 0.083437
Train Epoch: 45 [61440/388729 (16%)]	Loss: 0.080546
Train Epoch: 45 [71680/388729 (18%)]	Loss: 0.050747
Train Epoch: 45 [81920/388729 (21%)]	Loss: 0.077744
Train Epoch: 45 [92160/388729 (24%)]	Loss: 0.061656
Train Epoch: 45 [102400/388729 (26%)]	Loss: 0.050399
Train Epoch: 45 [112640/388729 (29%)]	Loss: 0.100423
Train Epoch: 45 [122880/388729 (32%)]	Loss: 0.056556
Train Epoch: 45 [133120/388729 (34%)]	Loss: 0.09400

Train Epoch: 48 [337920/388729 (87%)]	Loss: 0.054112
Train Epoch: 48 [348160/388729 (89%)]	Loss: 0.038784
Train Epoch: 48 [358400/388729 (92%)]	Loss: 0.039234
Train Epoch: 48 [368640/388729 (95%)]	Loss: 0.062084
Train Epoch: 48 [378880/388729 (97%)]	Loss: 0.038132

Validation set: Average loss: 0.0007, Accuracy: 3625/3870 (94%)

Train Epoch: 49 [0/388729 (0%)]	Loss: 0.045607
Train Epoch: 49 [10240/388729 (3%)]	Loss: 0.040138
Train Epoch: 49 [20480/388729 (5%)]	Loss: 0.049771
Train Epoch: 49 [30720/388729 (8%)]	Loss: 0.042345
Train Epoch: 49 [40960/388729 (11%)]	Loss: 0.066419
Train Epoch: 49 [51200/388729 (13%)]	Loss: 0.075483
Train Epoch: 49 [61440/388729 (16%)]	Loss: 0.074180
Train Epoch: 49 [71680/388729 (18%)]	Loss: 0.112404
Train Epoch: 49 [81920/388729 (21%)]	Loss: 0.070033
Train Epoch: 49 [92160/388729 (24%)]	Loss: 0.070003
Train Epoch: 49 [102400/388729 (26%)]	Loss: 0.078903
Train Epoch: 49 [112640/388729 (29%)]	Loss: 0.040999
Train Epoch: 49 [122880/388729 (32%)]	Loss: 0.09119

In [10]:
state_dict_1 = torch.load('Models/conv42_aug_100_100.pth')
model1 = Conv4Net().to(device)
model1.load_state_dict(state_dict_1)
model1.eval()

state_dict_2 = torch.load('Models/mcnn42_aug_500_100.pth')
model2 = MultiScaleCNN().to(device)
model2.load_state_dict(state_dict_2)
model2.eval()

state_dict_3 = torch.load('Models/amcnn1_32_100.pth')
model3 = AdaptiveMultiScaleCNN().to(device)
model3.load_state_dict(state_dict_3)
model3.eval()

state_dict_4 = torch.load('Models/amcnn1_50_50.pth')
model4 = AdaptiveMultiScaleCNN().to(device)
model4.load_state_dict(state_dict_4)
model4.eval()

state_dict_5 = torch.load('Models/amcnn1_70_50.pth')
model5 = AdaptiveMultiScaleCNN().to(device)
model5.load_state_dict(state_dict_5)
model5.eval()

exp_name = 'committee_3'

In [11]:
t0,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11 = get_transforms(32)
t20,t21,t22,t23,t24,t25,t26,t27,t28,t29,t210,t211 = get_transforms(50)
t30,t31,t32,t33,t34,t35,t36,t37,t38,t39,t310,t311 = get_transforms(70)

test_dir = 'images/test_images'

def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGB')

outfile = 'Output/'+exp_name+'_out.csv'

output_file = open(outfile, "w")
output_file.write("Filename,ClassId\n")
for f in tqdm(os.listdir(test_dir)):
    if 'ppm' in f:
        data1 = t211(pil_loader(test_dir + '/' + f))
        data2 = t21((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data3 = t22((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data4 = t23((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data5 = t24((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data6 = t25((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data7 = t26((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data8 = t27((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data9 = t28((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data10 = t29((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data11 = t210((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data12 = t20((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        
        final_input_1 = torch.cat((data1,data2,data3,data4,data5,data6,data7,data8,data9,data10,data11,data12),0)
         
        ntrans, c, h, w = final_input_1.size()
        temp_output_1 = model1(Variable(final_input_1.view(-1, c, h, w)).to(device))
        temp_output_2 = model2(Variable(final_input_1.view(-1, c, h, w)).to(device))
        temp_output_3 = model4(Variable(final_input_1.view(-1, c, h, w)).to(device))
        
        data21 = t11(pil_loader(test_dir + '/' + f))
        data22 = t1((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data23 = t2((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data24 = t3((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data25 = t4((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data26 = t5((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data27 = t6((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data28 = t7((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data29 = t8((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data210 = t9((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data211 = t10((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data212 = t0((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        
        final_input_2 = torch.cat((data21,data22,data23,data24,data25,data26,data27,data28,data29,data210,data211,data212),0)
         
        ntrans, c, h, w = final_input_2.size()
        temp_output_4 = model3(Variable(final_input_2.view(-1, c, h, w)).to(device))
        
        data31 = t311(pil_loader(test_dir + '/' + f))
        data32 = t31((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data33 = t32((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data34 = t33((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data35 = t34((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data36 = t35((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data37 = t36((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data38 = t37((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data39 = t38((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data310 = t39((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data311 = t310((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        data312 = t30((pil_loader(test_dir + '/' + f))).unsqueeze(0)
        
        final_input_3 = torch.cat((data31,data32,data33,data34,data35,data36,data37,data38,data39,data310,data311,data312),0)
         
        ntrans, c, h, w = final_input_3.size()
        temp_output_5 = model5(Variable(final_input_3.view(-1, c, h, w)).to(device))
        
        
        ensemble_res = torch.cat((temp_output_1, temp_output_2, temp_output_3,temp_output_4,temp_output_5),0)
        output = ensemble_res.mean(0)
        pred = output.data.max(0, keepdim=True)[1]
    
        file_id = f[0:5]
        output_file.write("%s,%d\n" % (file_id, pred))

output_file.close()

print("Succesfully wrote " + outfile + ', you can upload this file to the kaggle '
      'competition at https://www.kaggle.com/c/nyu-cv-fall-2018/')
        

100%|██████████| 12630/12630 [10:20<00:00, 20.34it/s]

Succesfully wrote Output/committee_3_out.csv, you can upload this file to the kaggle competition at https://www.kaggle.com/c/nyu-cv-fall-2018/


### This is my best prediction which got 99.287 on the Public leaderboard


Decided I'll have another go at this and train a ResNeXt to add to my ensemble

In [3]:
resnext = torchvision.models.resnext50_32x4d()
nclasses = 43
num_ftrs = resnext.fc.in_features
resnext.fc = torch.nn.Sequential(
    torch.nn.Linear(
        in_features=num_ftrs,
        out_features=nclasses
    ),
    nn.LogSoftmax()
)

resnext = resnext.to(device)

In [8]:
train_dl, val_dl = get_data(size=50)

In [ ]:
optimizer = torch.optim.Adam(resnext.parameters())
sched = CosineAnnealingWarmRestarts(optimizer, T_0=1)
train_loop(resnext, train_dl, val_dl, optimizer, sched, epochs=50, exp_name='resnext_50_')

/home/rm5310/pyenv/py3.6.3/lib/python3.6/site-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Train Epoch: 1 [0/388729 (0%)]	Loss: 1.214175
Train Epoch: 1 [10240/388729 (3%)]	Loss: 1.541363
Train Epoch: 1 [20480/388729 (5%)]	Loss: 1.271260
Train Epoch: 1 [30720/388729 (8%)]	Loss: 0.971884
Train Epoch: 1 [40960/388729 (11%)]	Loss: 0.730793
Train Epoch: 1 [51200/388729 (13%)]	Loss: 0.665881
Train Epoch: 1 [61440/388729 (16%)]	Loss: 0.773567
Train Epoch: 1 [71680/388729 (18%)]	Loss: 0.684050
Train Epoch: 1 [81920/388729 (21%)]	Loss: 0.547096
Train Epoch: 1 [92160/388729 (24%)]	Loss: 0.503170
Train Epoch: 1 [102400/388729 (26%)]	Loss: 0.485474
Train Epoch: 1 [112640/388729 (29%)]	Loss: 0.486393
Train Epoch: 1 [122880/388729 (32%)]	Loss: 0.388140
Train Epoch: 1 [133120/388729 (34%)]	Loss: 0.329865
Train Epoch: 1 [143360/388729 (37%)]	Loss: 0.337036
Train Epoch: 1 [153600/388729 (39%)]	Loss: 0.297575
Train Epoch: 1 [163840/388729 (42%)]	Loss: 0.287095
Train Epoch: 1 [174080/388729 (45%)]	Loss: 0.359121
Train Epoch: 1 [184320/388729 (47%)]	Loss: 0.306663
Train Epoch: 1 [194560/388729 